<a href="https://colab.research.google.com/github/nxxk23/NAKBID/blob/main/NINK/gradio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip -q install gradio requests

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 MB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.1/322.1 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.6/12.6 MB 71.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 4.2 MB/s eta 0:00:00


In [ ]:
import gradio as gr
import requests

# Function to call the FastAPI endpoint and get drug interactions
def get_interactions(drug_names, tpuid_ids):
    url = "http://quanta2.manageai.co.th:7777/interactions/"
    # Prepare the data to send to FastAPI
    data = {
        "input_text": drug_names.split(","),
        "tpu_id": [int(x) for x in tpuid_ids.split(",")]
    }
    response = requests.post(url, json=data)

    if response.status_code == 200:
        interactions_data = response.json().get("interactions", [])
        interaction_pairs = []
        # Prepare a list of drug pairs to show in the dropdown
        for interaction in interactions_data:
            pair_name = f"{interaction['drug_name1']} <-> {interaction['drug_name2']}"
            interaction_pairs.append(pair_name)  # Just names for dropdown
        return interaction_pairs, interactions_data  # Return pairs and full interaction data
    else:
        print(f"Error fetching data: {response.status_code}")
        return [], []

# Function to fetch details of a selected interaction
def get_interaction_details(selected_interaction, interaction_data):
    print(f"Selected interaction: {selected_interaction}")  # Log the selected interaction

    # Split the selected interaction into two drug names
    drug_name1, drug_name2 = selected_interaction.split(" <-> ")
    print(f"Drug 1: {drug_name1}, Drug 2: {drug_name2}")  # Log the split drug names

    # Find the details for the selected pair
    details = next((item for item in interaction_data
                    if item['drug_name1'] == drug_name1 and item['drug_name2'] == drug_name2), None)

    if details:
        print(f"Found details: {details}")  # Log the found details
        return {
            "Significance": details["Significance"],
            "Onset": details["Onset"],
            "Severity": details["Severity"],
            "Documentation": details["Documentation"],
            "Effect": details.get("effect", ""),
            "Mechanism": details.get("mechanism", ""),
            "Management": details.get("management", ""),
            "Discussion": details.get("discussion", "")
        }
    else:
        print("No details found for the selected interaction")  # Log if details aren't found
    return {}


# Define sample input sets for convenience
def set_sample_input(sample_set):
    samples = {
        "Set 1": ("AMILORIDE+HCTZ TAB., POTASSIUM CHLORIDE INJ 20MEQ/10ML,ASPIRIN 300 MG TAB,BEFARIN 3", "262926, 519545,226243,108159"),
        "Set 2": ("AMILORIDE+HCTZ TAB., POTASSIUM CHLORIDE INJ 20MEQ/10ML", "262926, 519545"),
        "Set 3": ("AMILORIDE+HCTZ TAB., POTASSIUM CHLORIDE INJ 20MEQ/10ML,ASPIRIN 300 MG TAB,BEFARIN 3", "262926, 519545, 226243, 108159"),
        "Set 4": ("AAAA,BBBB", "0000000, 0000000")  # No interactions expected
    }
    return samples.get(sample_set, ("", ""))

# Gradio UI Setup
with gr.Blocks() as demo:
    gr.Markdown("## 💊 Drug Interaction Checker")

    # Dropdown for selecting sample set
    sample_set_dropdown = gr.Dropdown(
        choices=["Select Sample Set", "Set 1", "Set 2", "Set 3", "Set 4"],
        label="Select Test Sample",
        interactive=True
    )

    # Inputs for drug names and TPUIDs
    with gr.Row():
        drug_input = gr.Textbox(label="Enter Drug Names (comma-separated)")
        tpu_input = gr.Textbox(label="Enter TPUIDs (comma-separated)")

    submit_btn = gr.Button("Check Interactions")

    # Dropdown for displaying interactions
    interaction_output = gr.Dropdown(
        label="Select Drug Interaction",
        choices=[],  # Start with an empty list
        interactive=True
    )

    # Textboxes for displaying details of the selected interaction
    with gr.Row():
        significance_box = gr.Textbox(label="Significance", interactive=False)
        onset_box = gr.Textbox(label="Onset", interactive=False)
        severity_box = gr.Textbox(label="Severity", interactive=False)
        documentation_box = gr.Textbox(label="Documentation", interactive=False)

    with gr.Accordion("Effect", open=True):
        effect_box = gr.Textbox(label="Effect", interactive=False, lines=1.5)

    with gr.Accordion("Mechanism", open=True):
        mechanism_box = gr.Textbox(label="Mechanism", interactive=False, lines=1.5)

    with gr.Accordion("Management", open=True):
        management_box = gr.Textbox(label="Management", interactive=False, lines=1.5)

    with gr.Accordion("Discussion", open=True):
        discussion_box = gr.Textbox(label="Discussion", interactive=False, lines=1.5)

    footer = gr.Markdown("")

    # Link sample set selection to the inputs
    sample_set_dropdown.change(
        set_sample_input,
        inputs=[sample_set_dropdown],
        outputs=[drug_input, tpu_input]
    )

        # Handle the submit button to fetch interactions
    def on_submit(drug_names, tpuid_ids):
        interactions, interaction_data = get_interactions(drug_names, tpuid_ids)
        print(f"Fetched interactions: {interactions}")  # Log fetched interactions
        print(f"Interaction Data: {interaction_data}")  # Log the entire interaction data for inspection

        if not interactions:
            return [], [], []  # Return empty dropdown if no interactions found
        return gr.update(choices=interactions), gr.update(value=interaction_data), gr.State(interaction_data)

    # Add an error output (gr.Textbox) for error messages
    error_message_box = gr.Textbox(label="Error", interactive=False, visible=False)

    submit_btn.click(
        on_submit,
        inputs=[drug_input, tpu_input],
        outputs=[interaction_output, error_message_box, gr.State()]
    )

    # When an interaction is selected, show details in the textboxes
    def on_interaction_select(selected_interaction, interaction_data):
        print(f"Selected interaction: {selected_interaction}")  # Log the selected interaction
        print(f"Interaction data available: {interaction_data}")  # Log the available interaction data

        if not interaction_data:
            return ["", "", "", "", "", "", "", ""]

        # Split the selected interaction to extract drug names
        try:
            drug_name1, drug_name2 = selected_interaction.split(" <-> ")
            print(f"Drug 1: {drug_name1.strip()}, Drug 2: {drug_name2.strip()}")  # Log the split drug names
        except Exception as e:
            print(f"Error splitting selected interaction: {e}")  # Log any error that happens during splitting
            return ["", "", "", "", "", "", "", ""]

        # Try to find the interaction details from the data
        details = next((item for item in interaction_data
                        if item['drug_name1'].strip() == drug_name1.strip() and item['drug_name2'].strip() == drug_name2.strip()), None)

        if details:
            print(f"Found details: {details}")  # Log the found details
            return (
                details.get("Significance", ""),
                details.get("Onset", ""),
                details.get("Severity", ""),
                details.get("Documentation", ""),
                details.get("Effect", ""),
                details.get("Mechanism", ""),
                details.get("Management", ""),
                details.get("Discussion", "")
            )
        else:
            print("No details found for the selected interaction")  # Log if details aren't found
        return ["", "", "", "", "", "", "", ""]


    # Update the interaction details on selection
    interaction_output.select(
        on_interaction_select,
        inputs=[interaction_output, gr.State()],
        outputs=[significance_box, onset_box, severity_box, documentation_box,
                 effect_box, mechanism_box, management_box, discussion_box]
    )

# Launch the Gradio interface
demo.launch(debug=True)


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://12b38f8ee61e1183a0.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Fetched interactions: ['AMILORIDE+HCTZ TAB. <-> POTASSIUM CHLORIDE INJ 20MEQ/10ML', 'WARFARIN 3 MG TAB <-> ASPIRIN 300 MG TAB']
Interaction Data: [{'tpuid1': 262926, 'tpuid2': 519545, 'drug_name1': 'AMILORIDE+HCTZ TAB.', 'drug_name2': 'POTASSIUM CHLORIDE INJ 20MEQ/10ML', 'interaction_detail': 'POTASSIUM SPARING DIURETICS will increase potassium retention and can produce severe hyperkalemia.', 'Onset': 2, 'Severity': 1, 'Documentation': 1, 'Significance': 1, 'management': 'Do not use this combination without documented evidence that a patient has clinical symptoms of hypokalemia unresponsive to either agent alone. If the combination is required, the patient should have strict dietary counseling and close monitoring of serum', 'mechanism': 'A reduction in renal elimination of the potassium ion.', 'discussion': 'One study shows death from hyperkalemia due to potassium supplementation to be one of the more common causes of drug related deaths.3 Studies have shown that potassium sparing diu